# Entrenamiento del modelo para el reconocimiento de Facturas

En este notebook, realizamos el entrenamiento de BERT (Bidirectional Encoder Representations from Transformers), un modelo para tareas de Procesamiento de Lenguaje Natural (NLP).

En este caso, lo estamos utilizando para tareas de etiquetado en un texto de entrada, que en este caso es el OCR de una factura. Esto nos permitirá extraer los datos para luego pasarlos automáticamente a un formato JSON y hacer el procesamiento de documentos mucho más accesible.


### Importación de Bibliotecas

- `import torch`: Importa la biblioteca PyTorch, un marco de trabajo de aprendizaje profundo.
- `from torch.utils.data import Dataset, DataLoader`: Importa la clase `Dataset` y `DataLoader` de PyTorch, que son útiles para manejar conjuntos de datos durante el entrenamiento.
- `from transformers import BertTokenizer, BertForTokenClassification, BertTokenizerFast`: Importa las clases y funciones necesarias de la biblioteca Transformers de Hugging Face, específicamente para el modelo BERT y su tokenizador.
- `from torch.optim import AdamW`: Importa el optimizador AdamW de PyTorch, que se utiliza comúnmente para optimizar los modelos de aprendizaje profundo.
- `from sklearn.model_selection import train_test_split`: Importa la función `train_test_split` de scikit-learn, que se utiliza para dividir un conjunto de datos en conjuntos de entrenamiento y prueba.
- `from sklearn.preprocessing import LabelEncoder`: Importa la clase `LabelEncoder` de scikit-learn, que se utiliza para codificar etiquetas de clase como valores numéricos.


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForTokenClassification, BertTokenizerFast
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


### Definición del Modelo
En este caso estamos usando la variante multilenguaje con distinción entre mayúsculas y minúsculas, esto nos puede ayudar con la detección de nombres que empiezan en mayúscula por ejemplo. Además definimos un codificador de etiquetas

In [2]:
MODEL_NAME = 'google-bert/bert-base-multilingual-cased'
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)
label_encoder = LabelEncoder()
labels = [
    "O",  # Para tokens que no son parte de ninguna entidad nombrada
    "B-invoice_id", "I-invoice_id",
    "B-issue_date", "I-issue_date",
    "B-due_date", "I-due_date",
    "B-issuer_name", "I-issuer_name",
    "B-issuer_address", "I-issuer_address",
    "B-issuer_phone",
    "B-issuer_email",
    "B-issuer_tax_id",
    "B-recipient_name", "I-recipient_name",
    "B-recipient_address", "I-recipient_address",
    "B-recipient_phone",
    "B-recipient_email",
    "B-recipient_tax_id",
    "B-item_description", "I-item_description",
    "B-item_quantity",
    "B-item_unit_price",
    "B-item_total",
    "B-subtotal",
    "B-tax_description", "I-tax_description",
    "B-tax_percentage",
    "B-tax_amount",
    "B-total",
    "B-payment_method",
    "UNK"
]

label_encoder.fit(labels)



LabelEncoder()

### Definición de Función para Alinear Tokens y Etiquetas

Se define una función llamada `align_tokens_and_labels` que toma como entrada un texto y las etiquetas asociadas. La función tokeniza el texto utilizando el tokenizador previamente definido, luego alinea las etiquetas con los tokens tokenizados y las convierte en valores numéricos utilizando el codificador de etiquetas.

### Definición de Clase `InvoiceDataset`

Se define una clase llamada `InvoiceDataset` que hereda de la clase `Dataset`. Esta clase se utiliza para representar un conjunto de datos de facturas. En el método `__init__`, se inicializan los textos y las etiquetas de las facturas, así como la longitud máxima permitida para el texto tokenizado. El método `__len__` devuelve la longitud del conjunto de datos, y el método `__getitem__` obtiene un ejemplo del conjunto de datos. Dentro de este método, se tokeniza el texto y se alinean las etiquetas utilizando la función `align_tokens_and_labels`, y luego se devuelve un diccionario con los IDs de entrada, la máscara de atención y las etiquetas, todos convertidos a tensores de PyTorch.


In [3]:
def align_tokens_and_labels(text,tags):
    encoded_input = tokenizer(text, is_split_into_words=True, padding="max_length", truncation=True, max_length=512)
    encoded_as_text = tokenizer.convert_ids_to_tokens(encoded_input["input_ids"])

    word_ids = encoded_input.word_ids()

    labels = []
    for i in range(len(word_ids)):
        if word_ids[i] is None:
            labels.append('UNK')
        else:
            labels.append(tags[word_ids[i]])
    
    labels = label_encoder.transform(labels)
    
    return {
        "encoded_input": encoded_input, 
        "encoded_labels": labels,
    }



class InvoiceDataset(Dataset):
    def __init__(self, texts, tags, max_len=512):
        self.texts = texts
        self.tags = tags
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        tags = self.tags[idx]


        # Tokenización y alineación de etiquetas
        result = align_tokens_and_labels(text, tags)
        encoding = result["encoded_input"]
        labels = result["encoded_labels"]

        return {
            'input_ids': torch.tensor(encoding['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(encoding['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

In [4]:


texts = []  # Lista de textos de factura
tags = []  # Lista de etiquetas (cada etiqueta es una lista de ids de etiquetas)

def load_tags(file_path):
    tags = []
    with open(file_path, 'r') as file:
        for line in file:
            # Dividir la línea por ' -> ' y tomar el segundo elemento, que es la etiqueta
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                tags.append(parts[1])  # Agrega la etiqueta a la lista
    return tags

def load_text(file_path):
    texts = []
    with open(file_path, 'r') as file:
        for line in file:
            # Dividir la línea por ' -> ' y tomar el segundo elemento, que es la etiqueta
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                texts.append(parts[0])  # Agrega la etiqueta a la lista
    return texts

tags = []
for i in range(1000):  # Ajusta el rango según la cantidad de facturas
    tags.append(load_tags(f'dataset_output/train/train{i}.tokens'))
    texts.append(load_text(f'dataset_output/train/train{i}.tokens'))

# Crear el dataset y dataloader
dataset = InvoiceDataset(texts, tags)
loader = DataLoader(dataset, batch_size=4, shuffle=True)



In [5]:
#Probar a tokenizar un texto para ver su longitud tokenizada

print(dataset[0]['input_ids'].tolist())
print(dataset[0]['labels'].tolist())

ids = tokenizer.convert_ids_to_tokens(dataset[0]['input_ids'])

for i in range(len(ids)):
    print(ids[i], label_encoder.inverse_transform([dataset[0]['labels'][i].item()]))

[101, 85245, 11465, 131, 108, 50461, 32168, 11305, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [6]:
# Dividir datos
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, tags, test_size=0.1)

# Crear DataLoaders
train_dataset = InvoiceDataset(train_texts, train_labels)
val_dataset = InvoiceDataset(val_texts, val_labels)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Determinar el dispositivo a usar (GPU si está disponible, de lo contrario CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modelo
model = BertForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(labels))
model.to(device)  # Mover el modelo a la GPU si está disponible

# Optimizador
optimizer = AdamW(model.parameters(), lr=5e-5)

# Función de entrenamiento
def train(model, dataloader, optimizer):
    model.train()
    total_loss = 0
    for batch in dataloader:
        # Mover los datos al dispositivo correcto
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)



Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
if torch.cuda.is_available():
    print("CUDA is available. GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")

CUDA is available. GPU: NVIDIA GeForce RTX 3060


In [8]:
def evaluate(model, dataloader):
    model.eval()  # Pone el modelo en modo evaluación
    total_accuracy = 0
    total_tokens = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            predictions = outputs.logits.argmax(dim=-1)  # Obtén la clase predicha para cada token

            # Ignorar tokens con etiquetas -100 (usadas en algunos datasets para ignorar ciertos tokens)
            mask = (labels != -100)
            correct_predictions = (predictions == labels) & mask
            total_accuracy += correct_predictions.sum().item()
            total_tokens += mask.sum().item()

    return total_accuracy / total_tokens


In [9]:
# Entrenamiento
for epoch in range(30):
    train_loss = train(model, train_loader, optimizer)
    val_accuracy = evaluate(model, val_loader)
    print(f'Epoch {epoch + 1}, Train Loss: {train_loss}, Val Accuracy: {val_accuracy:.2%}')

Epoch 1, Train Loss: 0.3050186497414798, Val Accuracy: 98.92%
Epoch 2, Train Loss: 0.020932163659944734, Val Accuracy: 99.96%
Epoch 3, Train Loss: 0.005056238773410761, Val Accuracy: 99.99%
Epoch 4, Train Loss: 0.0027228838473607876, Val Accuracy: 100.00%
Epoch 5, Train Loss: 0.0018284853508662993, Val Accuracy: 100.00%
Epoch 6, Train Loss: 0.001352987572575499, Val Accuracy: 99.99%
Epoch 7, Train Loss: 0.001185653043037231, Val Accuracy: 100.00%
Epoch 8, Train Loss: 0.0009847740071982278, Val Accuracy: 100.00%
Epoch 9, Train Loss: 0.0010203274901321702, Val Accuracy: 100.00%


KeyboardInterrupt: 

In [10]:
model.eval()  # Pon el modelo en modo evaluación
model.to(device)  # Asegúrate de que el modelo esté en el dispositivo correcto

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [11]:
from transformers import BertTokenizer

# Asumiendo que 'MODEL_NAME' es el nombre del modelo BERT que usaste
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

text = """
Factura
FH600301

Fecha de emisión: 2024-03-11
Fecha de vencimiento: 2024-04-15

Datos del Emisor: Datos del Receptor:
Williams, York and Schwartz Katrina Fritz
540 Strong Green North April, AK 08628 0936 Butler Villages Apt. 228 Williamhaven, NY 24061
2803178625 +1-905-539-8849x13583
hamptondanielle(O griffin.com mariah650 gmail.com
vsP770FmMb986 arF309WLD229
Descripción Cantidad Precio Unitario Total
expedite 24/7 systems 9 80.86 727.74
orchestrate web-enabled models 7 65.31 457.17
drive enterprise technologies 10 46.53 465.3

Subtotal: 1650.21
VAT (16%): 264.03
Total: 1914.24

Método de pago: PayPal
"""

encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
input_ids = encoded_input['input_ids'].to(device)
attention_mask = encoded_input['attention_mask'].to(device)


In [12]:
with torch.no_grad():  # No necesitas calcular gradientes aquí
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

In [13]:
import torch.nn.functional as F

# Aplicar softmax para obtener probabilidades
probabilities = F.softmax(logits, dim=-1)
predictions = torch.argmax(probabilities, dim=-1)
predicted_labels = [label_encoder.inverse_transform([label.item()])[0] for label in predictions[0]]
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

In [15]:


text = tokenizer.convert_tokens_to_string(tokens)
print(text)

def clean_token(tokens, labels):
    cleaned_tokens = []
    cleaned_labels = []
    for token, label in zip(tokens, labels):
        if token.startswith("##"):
            cleaned_tokens[-1] = cleaned_tokens[-1] +  token[2:]
        else:
            cleaned_tokens.append(token)
            cleaned_labels.append(label)
    return cleaned_tokens, cleaned_labels

def concat_consecutive_tags(tokens, labels):
    cleaned_tokens = []
    cleaned_labels = []
    for token, label in zip(tokens, labels):
        if len(cleaned_labels) > 0 and cleaned_labels[-1] == label:
            cleaned_tokens[-1] = cleaned_tokens[-1]  +  token
        else:
            cleaned_tokens.append(token)
            cleaned_labels.append(label)
    return cleaned_tokens, cleaned_labels

def concat_continue_labels_BI(tokens, labels):
    cleaned_tokens = []
    cleaned_labels = []
    for token, label in zip(tokens, labels):
        if len(cleaned_labels) > 0 and label.startswith("I-"):
            cleaned_tokens[-1] = cleaned_tokens[-1] + " " +  token
        else:
            cleaned_tokens.append(token)
            cleaned_labels.append(label)
    return cleaned_tokens, cleaned_labels

tokens, predicted_labels = clean_token(tokens, predicted_labels)
tokens, predicted_labels = concat_consecutive_tags(tokens, predicted_labels)
tokens, predicted_labels = concat_continue_labels_BI(tokens, predicted_labels)

for token, label in zip(tokens, predicted_labels):
    print(f'{token} -> {label}')


[CLS] Factura FH600301 Fechadeemisión: 2024-03-11 Fechadevencimiento: 2024-04-15 DatosdelEmisor:DatosdelReceptor: Williams, YorkandSchwartz KatrinaFritz 540StrongGreenNorthApril,AK086280936ButlerVillagesApt.228Williamhaven,NY24061 2803178625 + 1-905 -539-8849x13583 hamptondanielle(Ogriffin.commariah650gmail.com vsP770FmMb986arF309WLD229 DescripciónCantidadPrecioUnitarioTotal expedite 24/7systems 9 80.86 727.74 orchestrate web-enabledmodels 7 65.31 457.17 drive enterprisetechnologies 10 46.53 465.3 Subtotal: 1650.21 VAT (16%) : 264.03 Total: 1914.24 Métododepago: PayPal [SEP]
[CLS] -> UNK
Factura -> O
FH600301 -> B-invoice_id
Fechadeemisión: -> O
2024-03-11 -> B-issue_date
Fechadevencimiento: -> O
2024-04-15 -> B-due_date
DatosdelEmisor:DatosdelReceptor: -> O
Williams, YorkandSchwartz KatrinaFritz 540StrongGreenNorthApril,AK086280936ButlerVillagesApt.228Williamhaven,NY24061 -> B-issuer_name
2803178625 -> B-issuer_phone
+ -> B-recipient_phone
1-905 -> B-issuer_phone
-539-8849x13583 -> B-

In [16]:
torch.save(model, 'model.pth')
torch.save(model.state_dict(), 'model_state_dict.pth')